[[이유한님] 캐글 코리아 캐글 스터디 커널 커리큘럼](https://kaggle-kr.tistory.com/32)

[Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)

[Submarineering.EVEN BETTER PUBLIC SCORE until now.](https://www.kaggle.com/code/submarineering/submarineering-even-better-public-score-until-now)

This is **EVEN better the public score** kernel in the competition until now. I wanted strongly dedicate this kernel to that **scavenger colleagues** that spend the time tracking back the work of other members just to take chance and get any good idea only for his own benefit, but, however they don't share anything, just providing bad comments for those who share with the best will and without breaking the rules.

If you do not feel alluded, **please VOTE me UP**.

__DeepL 번역__  

이것은 지금까지의 경쟁에서 **공개 점수** 커널보다 훨씬 더 나은 커널입니다. 저는 이 커널을 다른 회원의 작업을 추적하여 기회를 잡고 자신의 이익을 위해서만 좋은 아이디어를 얻으려고 시간을 소비하지만, 아무것도 공유하지 않고 선한 의지로 규칙을 어기지 않고 공유하는 사람들에게 나쁜 댓글 만 제공하는 **청소부 동료**에게 강력하게 헌정하고 싶었습니다.

암시라고 느끼지 않으시면 **투표해 주세요**.

In [1]:
import os
import numpy as np
import pandas as pd
from subprocess import check_output
# print(check_output(["ls", "../input"]).decode("utf8"))  # 윈도우 사용으로 주석 처리

First thing first@

# Credits to the following awesome authors and kernels

Author: QuantScientist  
File: sub_200_ens_densenet.csv  
Link: https://www.kaggle.com/solomonk/pytorch-cnn-densenet-ensemble-lb-0-1538

Author: wvadim  
File: sub_TF_keras.csv  
Link: https://www.kaggle.com/wvadim/keras-tf-lb-0-18

Author: Ed Miller  
File: sub_fcn.csv  
Link: https://www.kaggle.com/bluevalhalla/fully-convolutional-network-lb-0-193

Author: Chia-Ta Tsai  
File: sub_blend009.csv  
Link: https://www.kaggle.com/cttsai/ensembling-gbms-lb-203

Author: DeveshMaheshwari  
File: sub_keras_beginner.csv  
Link: https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d

Author: Submarineering  
Files: submission38.csv , submission43.csv  
Link : https://www.kaggle.com/submarineering/submission38-lb01448

Without their truly dedicated efforts, this notebook will not be possible.

# Data Load


In [2]:
sub_path = "../input/statoil-iceverg-submissions"
sub_path = "./input/005_statoil-iceberg-classifier-challenge"   # 내 경로
all_files = os.listdir(sub_path)
all_files.append('submission38.csv')
all_files.append('submission43.csv')
all_files

['sample_submission.csv',
 'statoil-iceberg-classifier-challenge.zip',
 'test.json',
 'train.json',
 'submission38.csv',
 'submission43.csv']